# Let's load a dataset and try training!

In [21]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import sys

sys.path.append('../')
from scripts import forward_process as fp, reverse_process as rp, train
from scripts import util

In [12]:
ab1 = np.load("../data/image-colorization/ab/ab/ab1.npy")
ab2 = np.load("../data/image-colorization/ab/ab/ab2.npy")
ab3 = np.load("../data/image-colorization/ab/ab/ab3.npy")
l = np.load("../data/image-colorization/l/gray_scale.npy")

ab = np.concatenate((ab1,ab2,ab3),axis=0)

def createLab(n=100):
    lab = np.zeros((n,224,224,3))
    
    lab[:,:,:,0] = l[0:n:]
    lab[:,:,:,1:] = ab[0:n:]    
    
    return lab.astype("uint8")

## Let's load 1000 images

In [25]:
lab = createLab(1000)

# Resize to 64x64 for lower computational load
resized = []
for i in lab:
    img = cv2.normalize(i, None, alpha = -1, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    resized.append(cv2.resize(img, dsize=(64, 64), interpolation=cv2.INTER_CUBIC))

# Convert to tensor
lab = torch.tensor(np.array(resized))
dataloader = DataLoader(lab, batch_size=25, shuffle=True)

In [26]:
from pathlib import Path

def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr

results_folder = Path("../results")
results_folder.mkdir(exist_ok = True)
save_and_sample_every = 1000

In [ ]:
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Unet(
    dim=54,
    channels=channels,
    dim_mults=(1, 2, 4,)
)
model.to(device)

optimizer = Adam(model.parameters(), lr=1e-3)